In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

## Validation Strategies

In [2]:
#Objective Functions to minimize with Bayesian Optimization

# the isnan part is necessary because in the model-informed architecture when the ESN strays
# away from the attractor the trajectory may become unbounded


def SSV(x):
    # Single Shot Validation
    
    global rho, sigma_in, tikh_opt, k, ti
    rho      = 10**x[0]
    sigma_in = 10**x[1]
    
    lenn     = len(tikh)
    Mean     = np.zeros(lenn)
    
    print(ti - time.time())
    ti       = time.time()
        
    #Train
    Xa_train, Wout = train(U_washout, U_t, Y_t, tikh, sigma_in, rho)[:2]

    for j in range(lenn):
            #Validate
            Yh_val  = closed_loop(N_val, Xa_train[-1], Wout[j], sigma_in, rho)[0][1:]
            kh_val    = 0.5*np.linalg.norm(Yh_val, axis=1)**2
            Mean[j] = np.log10(np.mean((kh_val-k_v)**2))

            # prevent from diverging to infinity: put MSE equal to 10^10
            if np.isnan(Mean[j]) or np.isinf(Mean[j]):
                Mean[j] = 10

    a           = np.argmin(Mean)
    tikh_opt[k] = tikh[a]
    k          +=1
    print(k,Mean, Mean[a])
        
    return Mean[a]

def KFC(x):
    # chaotic K-Fold Validation
    
    global rho, sigma_in, tikh_opt, k, ti
    rho      = 10**x[0]
    sigma_in = 10**x[1]
        
    print(ti - time.time())
    ti       = time.time()
        
    lenn     = tikh.size
    Mean     = np.zeros(lenn)
    
    #Train using tv: training+val
    Xa_train, Wout, LHS0, RHS0 = train(U_washout, U_tv, Y_tv, tikh, sigma_in, rho)

    #Different Folds in the validation set
    for i in range(N_fo):

        p      = N_in + i*N_fw
        k_val  = 0.5*np.linalg.norm(UU[0,N_washout + p : N_washout + p + N_val], axis=1)**2

        #Train: remove the validation interval
        Xa1    = Xa_train[p+1:p+N_val+1]
        Y_t1   = Y_tv[0,p:p+N_val] #Xa_train and Y_tv indices are shifted by one

        LHS   = LHS0 - np.dot(Xa1.T, Xa1)
        RHS   = RHS0 - np.dot(Xa1.T, Y_t1)
        
        for j in range(lenn):
            Wout[j]  = np.linalg.solve(LHS + tikh[j]*np.eye(N_units+1), RHS)

            #Validate
            Yh_val    = closed_loop(N_val-1, Xa_train[p], Wout[j], sigma_in, rho)[0]
            kh_val    = 0.5*np.linalg.norm(Yh_val, axis=1)**2
    
            Mean[j]  += np.log10(np.mean((kh_val-k_val)**2))

        # prevent from diverging to infinity: put MSE equal to 10^10
        if np.isnan(Mean[j]) or np.isinf(Mean[j]):
            Mean[j] = 10*N_fo
                
    a           = np.argmin(Mean)
    tikh_opt[k] = tikh[a]
    k          +=1
    print(k,Mean/N_fo, Mean[a]/N_fo)

    return Mean[a]/N_fo


def RVC(x):
    # chaotic Recycle Validation
    
    global rho, sigma_in, tikh_opt, k, ti
    rho      = 10**x[0]
    sigma_in = 10**x[1]
        
    print(ti - time.time())
    ti       = time.time()
        
    lenn     = tikh.size
    Mean     = np.zeros(lenn)
    
    #Train using tv: training+val
    Xa_train, Wout, LHS0, RHS0 = train(U_washout, U_tv, Y_tv, tikh, sigma_in, rho)

    #Different Folds in the validation set
    for i in range(N_fo):

        p      = N_in + i*N_fw
        k_val  = 0.5*np.linalg.norm(UU[0,N_washout + p : N_washout + p + N_val], axis=1)**2

#         #Train: remove the validation interval
#         Xa1    = Xa_train[p+1:p+N_val+1]
#         Y_t1   = Y_tv[p:p+N_val] #Xa_train and Y_tv indices are shifted by one

#         LHS   = LHS0 - np.dot(Xa1.T, Xa1)
#         RHS   = RHS0 - np.dot(Xa1.T, Y_t1)
        
        for j in range(lenn):
#             Wout[j]  = np.linalg.solve(LHS + tikh[j]*np.eye(N_units+1), RHS)

            #Validate
            Yh_val    = closed_loop(N_val-1, Xa_train[p], Wout[j], sigma_in, rho)[0]
            kh_val    = 0.5*np.linalg.norm(Yh_val, axis=1)**2
    
            Mean[j]  += np.log10(np.mean((kh_val-k_val)**2))

        # prevent from diverging to infinity: put MSE equal to 10^10
        if np.isnan(Mean[j]) or np.isinf(Mean[j]):
            Mean[j] = 10*N_fo
                
    a           = np.argmin(Mean)
    tikh_opt[k] = tikh[a]
    k          +=1
    print(k,Mean/N_fo, Mean[a]/N_fo)

    return Mean[a]/N_fo   


In [ ]:
def Val_PI(x):
    # Single Shot Validation
    
    global rho, sigma_in, tikh_opt, k, ti
    
    rho      = 10**x[0]
    sigma_in = 10**x[1]
    
    lenn     = len(tikh)
    Mean     = np.zeros(lenn)
    
    print(ti - time.time())
    ti       = time.time()
        
    #Train
    Xa_train, Wout = train(U_washout, U_tv, Y_tv, tikh, sigma_in, rho)[:2]

    for j in range(lenn):
            #Validate
            Yh_val  = closed_loop(N_valPI, Xa_train[-1], Wout[j], sigma_in, rho)[0][1:]
            fh_val  = MFE(Yh_val)[:-1]
            dyh_val = (Yh_val[1:] - Yh_val[:-1])/dt
            Mean[j] = np.log10(np.mean((fh_val-dyh_val)**2))
            
            if Mean[j] < -3:
                plt.subplots(2,1)
                plt.subplot(211)
                plt.plot(Yh_val)
                print(0.5*np.linalg.norm(Yh_val[-1])**2)
                plt.subplot(212)
                plt.plot(MFE(Yh_val))
                plt.show()

            # prevent from diverging to infinity: put MSE equal to 10^10
            if np.isnan(Mean[j]) or np.isinf(Mean[j]):
                Mean[j] = 10

    a           = np.argmin(Mean)
    tikh_opt[k] = tikh[a]
    k          +=1
    print(k,Mean, Mean[a])
        
    return Mean[a]

def RVC_PI(x):
    # chaotic Recycle Validation
    
    global rho, sigma_in, tikh_opt, k, ti
    rho      = 10**x[0]
    sigma_in = 10**x[1]
        
    print(ti - time.time())
    ti       = time.time()
        
    lenn     = tikh.size
    Mean     = np.zeros(lenn)
    
    #Train using tv: training+val
    Xa_train, Wout, LHS0, RHS0 = train(U_washout, U_tv, Y_tv, tikh, sigma_in, rho)

    #Different Folds in the validation set
    for i in range(N_fo):

        p      = N_in + i*N_fw
#         k_val  = 0.5*np.linalg.norm(UU[0,N_washout + p : N_washout + p + N_val], axis=1)**2

#         #Train: remove the validation interval
#         Xa1    = Xa_train[p+1:p+N_val+1]
#         Y_t1   = Y_tv[p:p+N_val] #Xa_train and Y_tv indices are shifted by one

#         LHS   = LHS0 - np.dot(Xa1.T, Xa1)
#         RHS   = RHS0 - np.dot(Xa1.T, Y_t1)
        
        for j in range(lenn):
#             Wout[j]  = np.linalg.solve(LHS + tikh[j]*np.eye(N_units+1), RHS)

            #Validate
            Yh_val    = closed_loop(N_valPI, Xa_train[p], Wout[j], sigma_in, rho)[0]
            fh_val    = MFE(Yh_val)[:-1]
            dyh_val   = (Yh_val[1:] - Yh_val[:-1])/dt
            Mean[j]  += np.log10(np.mean((fh_val-dyh_val)**2))

        # prevent from diverging to infinity: put MSE equal to 10^10
        if np.isnan(Mean[j]) or np.isinf(Mean[j]):
            Mean[j] = 10*N_fo
                
    a           = np.argmin(Mean)
    tikh_opt[k] = tikh[a]
    k          +=1
    print(k,Mean/N_fo, Mean[a]/N_fo)

    return Mean[a]/N_fo

In [3]:
def MFE(q):
    """
    Defines the differential equations for Moehlis_2004

    Arguments:
        w :  vector of the state variables a_i
    """

    Ndim = 9
    # Problem Definition
    Lx = 4*math.pi
    Ly = 2.
    Lz = 2*math.pi
    Re = 400
    # Parameter values
    alfa  = 2*math.pi/Lx
    beta  = math.pi/2
    gamma = 2*math.pi/Lz

    k1 = np.sqrt(alfa**2 + gamma**2)
    k2 = np.sqrt(gamma**2 + beta**2)
    k3 = np.sqrt(alfa**2 + beta**2 + gamma**2) 
    
    a1, a2, a3, a4, a5, a6, a7, a8, a9 = q.T
    
    aa1 = beta**2/Re * (1. - a1) - np.sqrt(3/2)*beta*gamma/k3*np.multiply(a6,a8) + \
            np.sqrt(3/2)*beta*gamma/k2*np.multiply(a2,a3)

    aa2 = - ( 4/3*beta**2 + gamma**2) * a2/Re + 5/3*np.sqrt(2/3)*gamma**2/k1*np.multiply(a4,a6) - \
          gamma**2/np.sqrt(6)/k1*np.multiply(a5,a7) - \
          alfa*gamma*beta/np.sqrt(6)/k1/k3*np.multiply(a5,a8) - \
          np.sqrt(3/2)*beta*gamma/k2 * (np.multiply(a1,a3) + np.multiply(a3,a9))

    aa3 = - (beta**2 + gamma**2)/Re*a3 + 2/np.sqrt(6)*alfa*beta*gamma/k1/k2 * (np.multiply(a4,a7) + \
          np.multiply(a5,a6)) + \
          (beta**2*(3*alfa**2+gamma**2) - 3*gamma**2*(alfa**2+gamma**2))/np.sqrt(6)/k1/k2/k3*np.multiply(a4,a8)

    aa4 = - (3*alfa**2 + 4*beta**2)/3/Re*a4 - alfa/np.sqrt(6)*np.multiply(a1,a5) - \
         10/3/np.sqrt(6)*alfa**2/k1*np.multiply(a2,a6) - \
         np.sqrt(3/2)*alfa*beta*gamma/k1/k2*np.multiply(a3,a7) - \
         np.sqrt(3/2)*alfa**2*beta**2/k1/k2/k3*np.multiply(a3,a8) - \
         alfa/np.sqrt(6)*np.multiply(a5,a9)

    aa5 = - (alfa**2 + beta**2)/Re*a5 + alfa/np.sqrt(6)*np.multiply(a1,a4) + \
         alfa**2/np.sqrt(6)/k1*np.multiply(a2,a7) - \
         alfa*beta*gamma/np.sqrt(6)/k1/k3*np.multiply(a2,a8) + \
         alfa/np.sqrt(6)*np.multiply(a4,a9) + \
         2/np.sqrt(6)*alfa*beta*gamma/k1/k2*np.multiply(a3,a6)

    aa6 = - (3*alfa**2 + 4*beta**2 + 3*gamma**2)/3/Re*a6 + alfa/np.sqrt(6)*np.multiply(a1,a7) + \
         np.sqrt(3/2)*beta*gamma/k3*np.multiply(a1,a8) + \
         10/3/np.sqrt(6)*(alfa**2 - gamma**2)/k1*np.multiply(a2,a4) - \
         2*np.sqrt(2/3)*alfa*beta*gamma/k1/k2*np.multiply(a3,a5) + \
         alfa/np.sqrt(6)*np.multiply(a7,a9) + np.sqrt(3/2)*beta*gamma/k3*np.multiply(a8,a9)

    aa7 = - k3**2/Re*a7 - alfa/np.sqrt(6) * (np.multiply(a1,a6) + np.multiply(a6,a9)) + \
         (gamma**2 - alfa**2)/np.sqrt(6)/k1*np.multiply(a2,a5) + \
          alfa*beta*gamma/np.sqrt(6)/k1/k2*np.multiply(a3,a4)

    aa8 = - k3**2/Re*a8 + 2/np.sqrt(6)*alfa*beta*gamma/k3/k1*np.multiply(a2,a5) + \
         gamma**2*(3*alfa**2 - beta**2 + 3*gamma**2)/np.sqrt(6)/k1/k2/k3*np.multiply(a3,a4)
    
    aa9 = - 9*beta**2/Re*a9 + np.sqrt(3/2)*beta*gamma/k2*np.multiply(a2,a3) - \
         np.sqrt(3/2)*beta*gamma/k3*np.multiply(a6,a8)
    
    return np.stack([aa1,aa2,aa3,aa4,aa5,aa6,aa7,aa8,aa9],axis=1)

In [ ]:
def KFC_Phys(x):
    # chaotic K-Fold Validation
    
    global rho, sigma_in, tikh_opt, k
    rho      = x[0]
    sigma_in = x[1]
    
    lenn     = len(tikh)
    
    Mean     = np.zeros(lenn)
     
    #Train using tv: training+val
    Xa_train, Wout, LHS0, RHS0 = train(U_washout, U_tv, Y_tv, tikh)

    #Different Folds in the validation set
    for i in range(N_fo):

        p      = N_in + i*N_fw

        #Train: remove the validation interval
        Xa1    = Xa_train[p+1:p+N_val+1]
        Y_t1   = Y_tv[p:p+N_val] #Xa_train and Y_tv indices are shifted by one

        LHS   = LHS0 - np.dot(Xa1.T, Xa1)
        RHS   = RHS0 - np.dot(Xa1.T, Y_t1)
        
        for j in range(lenn):
            Wout[j]  = np.linalg.solve(LHS + tikh[j]*np.eye(N_units+1), RHS)

            #Validate
            Yh_val = closed_loop(N_phys, Xa_train[p], Wout[j])[0]
            dy_dt  = (Yh_val[1:] - Yh_val[:-1])/dt
            fy     = MFE(Yh_val[:-1])
            Mean[j]  += np.log10(np.mean((dy_dt - fy)**2))

        # prevent from diverging to infinity: put MSE equal to 10^10
        if np.isnan(Mean[j]) or np.isinf(Mean[j]):
            Mean[j] = 10*N_fo
    
    a           = np.argmin(Mean)
    tikh_opt[k] = tikh[a]
    k          +=1
    print(k,Mean/N_fo, Mean[a]/N_fo)

    return Mean[a]/N_fo

def KFC_MI(x):
    # chaotic K-Fold Validation
    
    global rho, sigma_in, tikh_opt, k
    rho      = x[0]
    sigma_in = x[1]
    
    lenn     = len(tikh)
    
    Mean     = np.zeros(lenn)
     
    #Train using tv: training+val
    Xa_train, Wout, LHS0, RHS0 = train(U_washout, U_tv, Y_tv, tikh)

    N_fo = 96  # number of folds
    N_in = 0   # interval before the first fold
    N_fw = N_lyap # how many steps forward the validation interval is shifted 

    #Different Folds in the validation set
    for i in range(N_fo):

        p      = N_in + i*N_fw
        Y_val  = U[N_washout + p : N_washout + p + N_val]

        #Train: remove the validation interval
        Xa1    = Xa_train[p+1:p+N_val+1]
        Y_t1   = Y_tv[p:p+N_val] #Xa_train and Y_tv indices are shifted by one

        LHS   = LHS0 - np.dot(Xa1.T, Xa1)
        RHS   = RHS0 - np.dot(Xa1.T, Y_t1)
        
        for j in range(lenn):
            Wout[j]  = np.linalg.solve(LHS + tikh[j]*np.eye(N_units+NPOD+1), RHS)

            #Validate
            Yh_val = closed_loop(N_val-1, Xa_train[p], Wout[j])[0]
#             Mean[j]  += np.mean((Yh_val-Y_val)**2)
            Mean[j]  += np.log10(np.mean((Yh_val-Y_val)**2))

        # prevent from diverging to infinity: put MSE equal to 10^10
        if np.isnan(Mean[j]) or np.isinf(Mean[j]):
            Mean[j] = 10*N_fo
    
    a           = np.argmin(Mean)
    tikh_opt[k] = tikh[a]
    k          +=1
    print(k,Mean/N_fo, Mean[a]/N_fo)

    return Mean[a]/N_fo
